In [1]:
!pip install -r ../requirements.txt
!pip install ipywidgets


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import os
import sys
directory_path = os.path.abspath(os.path.join('..'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from src import train_split, Scaler, model, RMSE

2022-11-25 15:26:44.001420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 15:26:44.181052: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-25 15:26:44.189620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-25 15:26:44.189648: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
df = pd.read_csv("../data/jobss-processed.csv")

In [9]:
target_col = "sal"
non_target_cols = df.drop(columns=target_col).columns
(train_data, train_labels), (test_data, test_labels) = train_split(df, target_col)

scale_cols = ["Longitude", "Latitude"]
standard_scaler = Scaler(train_data[scale_cols], test_data[scale_cols])
train_data[scale_cols] = standard_scaler.train_scale()
test_data[scale_cols] = standard_scaler.test_scale()

train_data = train_data.to_numpy()
test_data = test_data.to_numpy()

In [90]:
inputs = keras.Input(shape=(72,))
dense = layers.Dense(64, activation="relu")(inputs)
dense2 = layers.Dense(32, activation="relu")(dense)
dense3 = layers.Dense(16, activation="relu")(dense2)    
output = layers.Dense(1, activation="linear", name="Output/Prediction")(dense3)
model = keras.Model(inputs=inputs, outputs=output, name="Basic Regression Model")

In [91]:
model.summary()

Model: "Basic Regression Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 72)]              0         
                                                                 
 dense_53 (Dense)            (None, 64)                4672      
                                                                 
 dense_54 (Dense)            (None, 32)                2080      
                                                                 
 dense_55 (Dense)            (None, 16)                528       
                                                                 
 Output/Prediction (Dense)   (None, 1)                 17        
                                                                 
Total params: 7,297
Trainable params: 7,297
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.compile(optimizer=keras.optimizers.RMSprop(), loss=keras.losses.MeanAbsolutePercentageError(), metrics=["mae"])

In [148]:
model.fit(train_data, train_labels, epochs=150, batch_size=8)

Train on 378 samples
Epoch 1/150
378/378 [==============================] - 0s 294us/sample - loss: 10.6708 - mae: 442.1190
Epoch 2/150
378/378 [==============================] - 0s 282us/sample - loss: 10.5886 - mae: 435.8211
Epoch 3/150
378/378 [==============================] - 0s 272us/sample - loss: 10.4345 - mae: 435.4801
Epoch 4/150
378/378 [==============================] - 0s 260us/sample - loss: 10.5688 - mae: 440.3291
Epoch 5/150
378/378 [==============================] - 0s 260us/sample - loss: 10.6172 - mae: 442.3179
Epoch 6/150
378/378 [==============================] - 0s 266us/sample - loss: 10.4381 - mae: 436.3874
Epoch 7/150
378/378 [==============================] - 0s 265us/sample - loss: 10.5530 - mae: 438.8887
Epoch 8/150
378/378 [==============================] - 0s 246us/sample - loss: 10.5264 - mae: 434.3928
Epoch 9/150
378/378 [==============================] - 0s 264us/sample - loss: 10.4291 - mae: 433.8893
Epoch 10/150
378/378 [==============================

378/378 [==============================] - 0s 248us/sample - loss: 9.1908 - mae: 394.7127
Epoch 81/150
378/378 [==============================] - 0s 247us/sample - loss: 9.2901 - mae: 393.5429
Epoch 82/150
378/378 [==============================] - 0s 246us/sample - loss: 9.2504 - mae: 390.2951
Epoch 83/150
378/378 [==============================] - 0s 237us/sample - loss: 9.1142 - mae: 389.3118
Epoch 84/150
378/378 [==============================] - 0s 247us/sample - loss: 9.1377 - mae: 386.6616
Epoch 85/150
378/378 [==============================] - 0s 245us/sample - loss: 9.1063 - mae: 386.8908
Epoch 86/150
378/378 [==============================] - 0s 232us/sample - loss: 9.0740 - mae: 384.5661
Epoch 87/150
378/378 [==============================] - 0s 284us/sample - loss: 9.3424 - mae: 390.7852
Epoch 88/150
378/378 [==============================] - 0s 260us/sample - loss: 8.9721 - mae: 384.3220
Epoch 89/150
378/378 [==============================] - 0s 242us/sample - loss: 9.2565

In [149]:
y_pred = model.predict(test_data)

In [150]:
y_true = test_labels.to_numpy()

In [151]:
np.mean((y_true - y_pred) / y_true)

-0.17212351947567212

In [152]:
np.sqrt(np.mean(np.square(y_true - y_pred)))

2436.8677195978